# Training a model for catchment classification. 
* kategoryzacja zlewni oparta o większość dostępnych ceech zlewni z plików inp i rpt

# Imports

In [114]:
import swmmio
import pyswmm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing


desired_width = 500
pd.set_option("display.width", desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option("display.max_columns", 30)

In [115]:
classes = pd.DataFrame(
    data={
        "classes": [
            "compact_urban_development",
            "urban",
            "loose_urban_development",
            "wooded_area",
            "grassy",
            "loose_soil",
            "steep_area",
        ]
    }
)
classes

,classes
0,compact_urban_development
1,urban
2,loose_urban_development
3,wooded_area
4,grassy
5,loose_soil
6,steep_area


# Get files

In [116]:
INP_FILE_01 = "dataset/subcatchment_dataset_01.inp"
INP_FILE_02 = "dataset/subcatchment_dataset_02.inp"
INP_FILE_03 = "dataset/subcatchment_dataset_03.inp"
INP_FILE_04 = "dataset/subcatchment_dataset_04.inp"
INP_FILE_05 = "dataset/subcatchment_dataset_05.inp"
INP_FILE_06 = "dataset/subcatchment_dataset_06.inp"
INP_FILE_07 = "dataset/subcatchment_dataset_07.inp"
INP_FILE_08 = "dataset/subcatchment_dataset_08.inp"
INP_FILE_09 = "dataset/subcatchment_dataset_09.inp"
INP_FILE_10 = "dataset/subcatchment_dataset_10.inp"
files = [INP_FILE_01, INP_FILE_02, INP_FILE_03, INP_FILE_04, INP_FILE_05, INP_FILE_06, INP_FILE_07, INP_FILE_08, INP_FILE_09, INP_FILE_10]

# Run simulation

In [117]:
for f in files:
    with pyswmm.Simulation(f) as sim:
        for step in sim:
            pass

# Read inp and rpt file as swmmio model object

In [118]:
model_01 = swmmio.Model(INP_FILE_01)
model_02 = swmmio.Model(INP_FILE_02)
model_03 = swmmio.Model(INP_FILE_03)
model_04 = swmmio.Model(INP_FILE_04)
model_05 = swmmio.Model(INP_FILE_05)
model_06 = swmmio.Model(INP_FILE_06)
model_07 = swmmio.Model(INP_FILE_07)
model_08 = swmmio.Model(INP_FILE_08)
model_09 = swmmio.Model(INP_FILE_09)
model_10 = swmmio.Model(INP_FILE_10)

models = [model_01, model_02, model_03, model_04, model_05, model_06, model_07, model_08, model_09, model_10]

## Get subcatchments data from the model

In [219]:
raw_subcatchments_01 = model_01.subcatchments.dataframe
raw_subcatchments_02 = model_02.subcatchments.dataframe
raw_subcatchments_03 = model_03.subcatchments.dataframe
raw_subcatchments_04 = model_04.subcatchments.dataframe
raw_subcatchments_05 = model_05.subcatchments.dataframe
raw_subcatchments_06 = model_06.subcatchments.dataframe
raw_subcatchments_07 = model_07.subcatchments.dataframe
raw_subcatchments_08 = model_08.subcatchments.dataframe
raw_subcatchments_09 = model_09.subcatchments.dataframe
raw_subcatchments_10 = model_10.subcatchments.dataframe

subcatchments_01 = raw_subcatchments_01.copy()
subcatchments_02 = raw_subcatchments_02.copy()
subcatchments_03 = raw_subcatchments_03.copy()
subcatchments_04 = raw_subcatchments_04.copy()
subcatchments_05 = raw_subcatchments_05.copy()
subcatchments_06 = raw_subcatchments_06.copy()
subcatchments_07 = raw_subcatchments_07.copy()
subcatchments_08 = raw_subcatchments_08.copy()
subcatchments_09 = raw_subcatchments_09.copy()
subcatchments_10 = raw_subcatchments_10.copy()

subcatchments = [subcatchments_01, subcatchments_02, subcatchments_03, subcatchments_04, subcatchments_05, subcatchments_06, subcatchments_07, subcatchments_08, subcatchments_09, subcatchments_10]

### Drop unused columns

In [220]:
for frame in subcatchments:
    frame.drop(['N-Imperv', 'N-Perv', 'S-Imperv',	'S-Perv', 'coords', 'RouteTo', 'CurbLength', 'Outlet', 'Raingage', 'TotalRunon', 'TotalEvap', 'TotalInfil', 'ImpervRunoff', 'PervRunoff', 'TotalRunoffIn'], axis=1, inplace=True)

In [221]:
subcatchments_01.iloc[:5, :]

,Area,PercImperv,Width,PercSlope,PctZero,TotalPrecip,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,
S1,0.50,20.00,300.00,5.00,70,10.10,0.02,0.02,0.407
S10,1.87,45.00,136.75,15.00,90,10.10,0.11,0.09,0.587
S100,1.71,45.00,130.77,10.00,90,10.10,0.10,0.08,0.581
S1000,0.68,10.00,82.46,5.09,10,10.10,0.02,0.01,0.224
S1001,0.26,83.33,50.99,45.00,80,10.10,0.02,0.02,0.879


## Get categories

In [222]:
categories_01 = model_01.inp.tags
categories_02 = model_02.inp.tags
categories_03 = model_03.inp.tags
categories_04 = model_04.inp.tags
categories_05 = model_05.inp.tags
categories_06 = model_06.inp.tags
categories_07 = model_07.inp.tags
categories_08 = model_08.inp.tags
categories_09 = model_09.inp.tags
categories_10 = model_10.inp.tags
categories_01

,Name,Tag
ElementType,,
Subcatch,S1,loose_soil
Subcatch,S2,compact_urban_development
Subcatch,S3,loose_soil
Subcatch,S4,wooded_area
Subcatch,S5,loose_urban_development
...,...,...
Subcatch,S997,grassy
Subcatch,S998,steep_area
Subcatch,S999,urban


### Add categories column to subcatchments DataFrame

In [223]:
def merge_tag(subcatchment, model):
    subcatchment.reset_index(inplace=True)
    merged = subcatchment.merge(model.inp.tags, left_on="Name", right_on="Name", how="left")
    merged.rename(columns={"Tag": "categories"}, inplace=True)
    merged.set_index("Name", inplace=True)
    return merged

merged_df_01 = merge_tag(subcatchments_01, model_01)
merged_df_02 = merge_tag(subcatchments_02, model_02)
merged_df_03 = merge_tag(subcatchments_03, model_03)
merged_df_04 = merge_tag(subcatchments_04, model_04)
merged_df_05 = merge_tag(subcatchments_05, model_05)
merged_df_06 = merge_tag(subcatchments_06, model_06)
merged_df_07 = merge_tag(subcatchments_07, model_07)
merged_df_08 = merge_tag(subcatchments_08, model_08)
merged_df_09 = merge_tag(subcatchments_09, model_09)
merged_df_10 = merge_tag(subcatchments_10, model_10)

merged = [merged_df_01, merged_df_02, merged_df_03, merged_df_04, merged_df_05, merged_df_06, merged_df_07, merged_df_08, merged_df_09, merged_df_10]

df = pd.concat(merged)
df = df.reset_index(drop=True)

In [224]:
df.head()

,Area,PercImperv,Width,PercSlope,PctZero,TotalPrecip,TotalRunoffMG,PeakRunoff,RunoffCoeff,categories
0,0.50,20.00,300.00,5.00,70,10.10,0.02,0.02,0.407,loose_soil
1,1.87,45.00,136.75,15.00,90,10.10,0.11,0.09,0.587,compact_urban_development
2,1.71,45.00,130.77,10.00,90,10.10,0.10,0.08,0.581,compact_urban_development
3,0.68,10.00,82.46,5.09,10,10.10,0.02,0.01,0.224,grassy
4,0.26,83.33,50.99,45.00,80,10.10,0.02,0.02,0.879,steep_area


### Split data into features and target

In [226]:
X = df.drop('categories', axis=1)
y = df['categories']

X["TotalPrecip"] = pd.to_numeric(X["TotalPrecip"])
# X["TotalRunon"] = pd.to_numeric(X["TotalRunon"])
# X["TotalEvap"] = pd.to_numeric(X["TotalEvap"])
# X["TotalInfil"] = pd.to_numeric(X["TotalInfil"])
# X["PervRunoff"] = pd.to_numeric(X["PervRunoff"])
# X["ImpervRunoff"] = pd.to_numeric(X["ImpervRunoff"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(8008, 9) (8008, 7)
(2002, 9) (2002, 7)


In [227]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8008 entries, 2270 to 7270
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Area           8008 non-null   float64
 1   PercImperv     8008 non-null   float64
 2   Width          8008 non-null   float64
 3   PercSlope      8008 non-null   float64
 4   PctZero        8008 non-null   int64  
 5   TotalPrecip    8008 non-null   float64
 6   TotalRunoffMG  8008 non-null   float64
 7   PeakRunoff     8008 non-null   float64
 8   RunoffCoeff    8008 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 625.6 KB


In [228]:
X_train[:50]

,Area,PercImperv,Width,PercSlope,PctZero,TotalPrecip,TotalRunoffMG,PeakRunoff,RunoffCoeff
2270,0.02,10.00,14.14,20.00,10,38.33,0.01,0.01,0.825
8668,1.87,10.00,136.75,20.00,10,12.67,0.06,0.01,0.256
8293,0.81,10.00,90.00,20.00,10,12.67,0.03,0.01,0.280
7868,1.46,10.00,120.83,5.09,10,9.50,0.01,0.00,0.101
1095,1.61,5.09,126.89,15.00,10,20.00,0.19,0.14,0.590
3131,0.58,15.00,76.16,30.00,5,15.00,0.02,0.01,0.259
8116,1.96,10.00,140.00,5.09,10,12.67,0.05,0.01,0.214
1561,0.89,5.09,94.34,15.00,10,20.00,0.11,0.09,0.606
1670,1.52,65.00,123.29,61.67,80,20.00,0.27,0.21,0.879
9951,1.68,45.00,129.61,5.09,90,0.32,0.00,0.00,0.403


In [229]:
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(X_train))


# Build the model

In [244]:
model = Sequential()
model.add(normalizer)
model.add(Dense(units=X_train.shape[1], activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(18, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1], activation='softmax'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_8 (Normalizat  (None, 9)                19        
 ion)                                                            
                                                                 
 dense_21 (Dense)            (None, 9)                 90        
                                                                 
 dropout_8 (Dropout)         (None, 9)                 0         
                                                                 
 dense_22 (Dense)            (None, 18)                180       
                                                                 
 dropout_9 (Dropout)         (None, 18)                0         
                                                                 
 dense_23 (Dense)            (None, 7)                 133       
                                                      

In [245]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=100,
    validation_data=(X_test, y_test),
    validation_split=0.2,
    verbose=1,
    batch_size=16,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10)]
)

Epoch 1/100
501/501 [==============================] - 2s 3ms/step - loss: 1.4318 - accuracy: 0.4562 - val_loss: 1.0165 - val_accuracy: 0.5914
Epoch 2/100
501/501 [==============================] - 1s 3ms/step - loss: 0.9901 - accuracy: 0.5912 - val_loss: 0.7954 - val_accuracy: 0.6703
Epoch 3/100
501/501 [==============================] - 1s 2ms/step - loss: 0.8279 - accuracy: 0.6475 - val_loss: 0.6443 - val_accuracy: 0.7328
Epoch 4/100
501/501 [==============================] - 1s 3ms/step - loss: 0.7307 - accuracy: 0.6733 - val_loss: 0.5244 - val_accuracy: 0.7692
Epoch 5/100
501/501 [==============================] - 1s 3ms/step - loss: 0.6489 - accuracy: 0.7022 - val_loss: 0.4383 - val_accuracy: 0.9081
Epoch 6/100
501/501 [==============================] - 1s 3ms/step - loss: 0.5855 - accuracy: 0.7378 - val_loss: 0.3648 - val_accuracy: 0.9171
Epoch 7/100
501/501 [==============================] - 1s 3ms/step - loss: 0.5329 - accuracy: 0.7670 - val_loss: 0.3053 - val_accuracy: 0.9291

In [246]:
# Evaluate ANN model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test set accuracy: {accuracy:.3f}")

63/63 [==============================] - 0s 2ms/step - loss: 0.0133 - accuracy: 1.0000
Test set accuracy: 1.000


In [247]:
# model.predict(X_test)
y_pred = model.predict(X_test)

63/63 [==============================] - 0s 1ms/step


In [248]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics

,loss,accuracy,val_loss,val_accuracy,epoch
0,1.431812,0.456169,1.016533,0.591409,0
1,0.990071,0.591159,0.795357,0.670330,1
2,0.827873,0.647478,0.644294,0.732767,2
3,0.730679,0.673327,0.524435,0.769231,3
4,0.648929,0.702173,0.438275,0.908092,4
...,...,...,...,...,...
70,0.086860,0.972777,0.011039,1.000000,70
71,0.083363,0.972652,0.015176,1.000000,71
72,0.086937,0.970155,0.015461,1.000000,72
73,0.081447,0.973402,0.008028,1.000000,73


# Model evaluate

In [249]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['accuracy'], name='accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['loss'], name='loss'), row=1, col=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_accuracy'], name='val_accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_loss'], name='val_loss'), row=1, col=2)

fig.update_xaxes(title_text='epochs')
fig.update_yaxes(title_text='accuracy', row=1, col=1)
fig.update_yaxes(title_text='loss', row=1, col=2)
fig.update_layout(width=1000, title='Accuracy and Loss')
fig.show()

In [250]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(test_acc)

1.0


Predykcja na podstawie modelu.
* model.evaluate(y_true, y_pred) - pozwala obliczyć metryki modelu
* model.predict_classes() - pozwala zwrócić odpowiednio przewidziane klasy
* model.predict_proba(), model.predict() - pozwala zwrócić prawdopodobieństwo danej klasy

In [251]:
predictions = model.predict(X_test)
predictions

63/63 [==============================] - 0s 2ms/step


array([[0.0000000e+00, 1.0000000e+00, 2.1441476e-13, ..., 0.0000000e+00, 0.0000000e+00, 5.1129209e-23],
       [0.0000000e+00, 1.0000000e+00, 3.7038948e-12, ..., 0.0000000e+00, 0.0000000e+00, 4.8140197e-24],
       [3.3895907e-03, 1.0858879e-09, 3.8966184e-05, ..., 6.1264720e-05, 9.9453652e-01, 2.7511138e-12],
       ...,
       [0.0000000e+00, 9.9999416e-01, 5.8327605e-06, ..., 1.5853244e-33, 5.9536665e-35, 2.8382350e-11],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [3.7502542e-13, 2.1413781e-03, 3.0245493e-36, ..., 6.9643829e-07, 0.0000000e+00, 9.9785787e-01]], dtype=float32)

In [252]:
predictions_df = pd.DataFrame(predictions)
predictions_df

,0,1,2,3,4,5,6
0,0.000000e+00,1.000000e+00,2.144148e-13,0.000000e+00,0.000000e+00,0.000000e+00,5.112921e-23
1,0.000000e+00,1.000000e+00,3.703895e-12,0.000000e+00,0.000000e+00,0.000000e+00,4.814020e-24
2,3.389591e-03,1.085888e-09,3.896618e-05,1.973641e-03,6.126472e-05,9.945365e-01,2.751114e-12
3,7.702171e-15,1.291682e-04,1.397327e-32,0.000000e+00,1.694242e-03,0.000000e+00,9.981766e-01
4,2.164604e-03,2.557837e-11,1.103415e-05,1.046236e-03,4.143651e-05,9.967366e-01,3.195477e-14
...,...,...,...,...,...,...,...
1997,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
1998,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
1999,0.000000e+00,9.999942e-01,5.832761e-06,1.332117e-35,1.585324e-33,5.953667e-35,2.838235e-11
2000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00


In [253]:
predictions_cls = predictions.argmax(axis=-1)
predictions_cls

array([1, 1, 5, ..., 1, 4, 6], dtype=int64)

# Zapisanie modelu

In [254]:
model.save(r'C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\all_columns')

INFO:tensorflow:Assets written to: C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\all_columns\assets


# Załadowanie modelu

In [255]:
from tensorflow import keras
model = keras.models.load_model(r'C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\all_columns')

In [256]:
X_test

,Area,PercImperv,Width,PercSlope,PctZero,TotalPrecip,TotalRunoffMG,PeakRunoff,RunoffCoeff
7438,1.43,5.09,119.58,10.00,10,9.50,0.01,0.00,0.063
9058,1.00,5.09,100.00,10.00,10,0.32,0.00,0.00,0.005
568,0.77,30.00,87.75,15.00,80,10.10,0.04,0.03,0.462
9344,0.60,15.00,77.46,30.00,5,0.32,0.00,0.00,0.007
2236,1.39,30.00,117.90,5.09,80,38.33,0.44,0.30,0.820
...,...,...,...,...,...,...,...,...,...
533,0.19,83.33,43.59,45.00,80,10.10,0.02,0.01,0.880
132,1.36,65.00,116.62,61.67,80,10.10,0.10,0.09,0.761
7329,0.30,10.00,54.77,20.00,10,9.50,0.00,0.00,0.127
8466,1.25,65.00,111.80,61.67,80,12.67,0.12,0.04,0.772


In [257]:
pred = model.predict(X_test)
pred

63/63 [==============================] - 0s 2ms/step


array([[0.0000000e+00, 1.0000000e+00, 2.1441476e-13, ..., 0.0000000e+00, 0.0000000e+00, 5.1129209e-23],
       [0.0000000e+00, 1.0000000e+00, 3.7038948e-12, ..., 0.0000000e+00, 0.0000000e+00, 4.8140197e-24],
       [3.3895907e-03, 1.0858879e-09, 3.8966184e-05, ..., 6.1264720e-05, 9.9453652e-01, 2.7511138e-12],
       ...,
       [0.0000000e+00, 9.9999416e-01, 5.8327605e-06, ..., 1.5853244e-33, 5.9536665e-35, 2.8382350e-11],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [3.7502542e-13, 2.1413781e-03, 3.0245493e-36, ..., 6.9643829e-07, 0.0000000e+00, 9.9785787e-01]], dtype=float32)